# Artificial Neural Network Techniques

Developed by: David

---

Implementation of neural network architecture for deep learning applications, adaptable for various types of data.

---

# Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
tf.__version__

# Data Preprocessing

## Importing the dataset

In [ ]:
dataset = pd.read_csv('data/default_credit_card_clients.txt', delimiter='\t',skiprows=1)
X = dataset.iloc[:, 1:-1].values # All columns except the first and the last
y = dataset.iloc[:, -1].values # Only the last column

In [ ]:
print(X)

In [ ]:
print(y)

## Encode the categorical variables

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# transformed_data = column_transformer.fit_transform(your_dataframe)

ct = ColumnTransformer(
    transformers=[
        ('gender_encoder', OneHotEncoder(), [1]),  # Encode the Gender variable
        ('education_encoder', OneHotEncoder(), [2]),  # Encode the Education variable
        ('marital_encoder', OneHotEncoder(), [3])  # Encode the Marital Status variable
    ],
    remainder='passthrough'  # Keep the rest of the columns as they are
)

# Apply the ColumnTransformer to X
X = np.array(ct.fit_transform(X))

## Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

## Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the ANN

## Initializing the ANN

In [ ]:
ann = tf.keras.models.Sequential()

## Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))

## Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=15, activation='relu'))

## Adding the output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Training the ANN

## Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Training the ANN on the Training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

## Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Making the Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
cm_ann_score = accuracy_score(y_test, y_pred)

## Applying k-Fold Cross Validation

In [ ]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score

# Wrap the model with KerasClassifier
ann_k = KerasClassifier(model=ann, epochs=100, batch_size=10, verbose=1)
accuracies = cross_val_score(estimator = ann_k, X = X_train, y = y_train, cv = 10)
kf_ann_accuracy = "{:.2f} %".format(accuracies.mean()*100)
kf_ann_sd = "{:.2f} %".format(accuracies.std()*100)

# Selecting the best model

In [ ]:
data = {'Model': ['ANN'],
        'CM Score': [cm_ann_score],
        'k-Fold Accuracy': [kf_ann_accuracy],
        'k-Fold SD': [kf_ann_sd]}

df = pd.DataFrame(data)

print(df)